# Epidemiology of Sedation in Mechanical Ventilation

## Init

In [1]:
import os
os.chdir("..")
os.getcwd()

'/Users/wliao0504/code/clif/CLIF-epi-of-sedation'

### Import

In [ ]:
from importlib import reload
import pandas as pd
import duckdb
from utils import pyCLIF as pc
reload(pc)
from utils.waterfall import process_resp_support_waterfall
import ipytest
import tableone
from utils.data_cleaner import remove_outliers_with_timing

Loaded configuration from config.json
Loaded configuration from config.json


In [186]:
helper = pc.load_config()
site = helper['site_name'].lower()
print(f"your site name is: {site}")

Loaded configuration from config.json
your site name is: mimic


## Cohort Identification

In [3]:
adt = pc.load_data("clif_adt")
hospitalization = pc.load_data("clif_hospitalization")

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_adt.parquet
Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_hospitalization.parquet


### Create ICU-stay level unique id

In [4]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    stitched_encounters = pc.stitch_encounters(hospitalization, adt)

In [5]:
# create a mapping table
query = """
SELECT DISTINCT patient_id, hospitalization_id, encounter_block
FROM stitched_encounters
"""
hosp_to_enc_blk_mapper = duckdb.sql(query).to_df()

In [6]:
query = """
SELECT hospitalization_id
    , encounter_block
    , date_trunc('hour', in_dttm) as in_date_hr
    , 1 as new_icu_stay
FROM stitched_encounters
WHERE location_category = 'icu'
"""
new_icu_start_hours = duckdb.sql(query).to_df()

hosp_ids_w_icu_stays = new_icu_start_hours['hospitalization_id'].unique().tolist()

### Hour 24 and 72

In [200]:
resp = pc.load_data(
    table = "clif_respiratory_support",
    filters = {
        "hospitalization_id": hosp_ids_w_icu_stays
    }
    )

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_respiratory_support.parquet


In [8]:
# resp_f = process_resp_support_waterfall(resp)
resp_f = pd.read_parquet("resp_f.parquet")

In [9]:
focal_hosp_ids = [
    '21738444', 
    '20004088', 
    '20006154', 
    '20018306'
    ]

In [10]:
# query = f"""
# SELECT SUM(CASE WHEN location_category = 'icu' THEN 1 ELSE 0 END) as total_icu_stays
# FROM adt
# WHERE hospitalization_id IN ({",".join(focal_hosp_ids)})
# """
# duckdb.sql(query).to_df()

In [11]:
resp_f['date_hr'] = resp_f['recorded_dttm'].dt.floor('h')

query = f"""
SELECT t1.hospitalization_id
    , t3.encounter_block
    , t1.date_hr
    , MAX(CASE WHEN t2.new_icu_stay = 1 THEN 1 ELSE 0 END) as new_icu_start_from_adt
    , MAX(CASE WHEN t1.device_category = 'imv' THEN 1 ELSE 0 END) as on_imv
    , MAX(CASE WHEN t1.tracheostomy is True OR t1.tracheostomy = 1 THEN 1 ELSE 0 END) as trach_ever
    , ROW_NUMBER() OVER (PARTITION BY t1.hospitalization_id ORDER BY t1.date_hr) as rn_by_hosp
    , CASE WHEN (
        rn_by_hosp = 1 -- new hospitalization
        OR new_icu_start_from_adt = 1 -- new icu stay
    ) THEN 1 ELSE 0 END as new_icu_stay
FROM resp_f as t1
LEFT JOIN new_icu_start_hours AS t2
    ON t1.hospitalization_id = t2.hospitalization_id
    AND t1.date_hr = t2.in_date_hr
LEFT JOIN hosp_to_enc_blk_mapper AS t3
    ON t1.hospitalization_id = t3.hospitalization_id
-- WHERE t1.hospitalization_id IN ({",".join(focal_hosp_ids)})
GROUP BY t1.hospitalization_id, t1.date_hr, t3.encounter_block
ORDER BY t1.hospitalization_id, t1.date_hr
"""
df1 = duckdb.sql(query).to_df()

In [12]:
query = """
-- generate unique icu stay ids
WITH t1 AS (
    SELECT hospitalization_id
        , encounter_block
        , date_hr
        , on_imv
        , new_icu_stay
        , SUM(new_icu_stay) OVER (ORDER BY hospitalization_id, date_hr) as icu_stay_id
    FROM df1
    -- keep only hospitalizations that have at least one hour on imv and no tracheostomy
    WHERE hospitalization_id IN (
        SELECT DISTINCT hospitalization_id
        FROM df1
        GROUP BY hospitalization_id
        HAVING MAX(on_imv) = 1 AND MAX(trach_ever) = 0
    )
),
-- generate unique imv streak ids
t2 AS (
    SELECT hospitalization_id
        , icu_stay_id
        -- , encounter_block
        , date_hr
        , on_imv
        , ROW_NUMBER() OVER (PARTITION BY icu_stay_id ORDER BY date_hr) as rn_overall
        , ROW_NUMBER() OVER (PARTITION BY icu_stay_id, on_imv ORDER BY date_hr) as rn_by_imv_status
        , rn_overall - rn_by_imv_status as imv_streak_id
    FROM t1
    -- keep only icu stays that have at least one hour on imv
    WHERE icu_stay_id IN (
        SELECT DISTINCT icu_stay_id
        FROM t1
        GROUP BY icu_stay_id
        HAVING MAX(on_imv) = 1
    )
    ORDER BY hospitalization_id, icu_stay_id, date_hr
),
-- mark the 24th and 72th hour of each imv streak
t3 AS (
    SELECT hospitalization_id, icu_stay_id, date_hr
        , imv_streak_id, on_imv
        , SUM(on_imv) OVER (PARTITION BY icu_stay_id, imv_streak_id ORDER BY date_hr) as imv_hrs_in_streak
        , CASE WHEN (imv_hrs_in_streak = 24) THEN 1 ELSE 0 END as hr_24_on_imv
        , CASE WHEN (imv_hrs_in_streak = 72) THEN 1 ELSE 0 END as hr_72_on_imv
        -- calculate hour since first intubation within each icu stay
        , MIN(CASE WHEN on_imv = 1 THEN date_hr END) OVER (PARTITION BY icu_stay_id) as first_imv_hr_in_icu_stay
        -- can only calculate diff in secs, so convert to hrs
        ,  EXTRACT(EPOCH FROM (date_hr - first_imv_hr_in_icu_stay)) / 3600 + 1 as hrs_since_first_imv
    FROM t2
    ORDER BY hospitalization_id, icu_stay_id, date_hr
    )
-- exclude cases with reintubation within 72 hours
SELECT hospitalization_id, icu_stay_id, date_hr
    , imv_streak_id, on_imv, imv_hrs_in_streak, hrs_since_first_imv
    , hr_24_on_imv, hr_72_on_imv
    , COUNT(DISTINCT CASE WHEN hrs_since_first_imv BETWEEN 0 AND 72 THEN imv_streak_id END) 
        OVER (PARTITION BY icu_stay_id) as n_imv_streaks_in_72_hrs
    , CASE WHEN n_imv_streaks_in_72_hrs <= 2 AND hr_24_on_imv = 1 THEN 1 ELSE 0 END as hr_24_on_imv_noreintub
    , CASE WHEN n_imv_streaks_in_72_hrs = 1 AND hr_72_on_imv = 1 THEN 1 ELSE 0 END as hr_72_on_imv_noreintub
FROM t3
ORDER BY hospitalization_id, icu_stay_id, date_hr
"""
df2 = duckdb.sql(query).to_df()

In [ ]:
# %%ipytest

# # sanity tests against the MIMIC-IV data
# @pytest.mark.parametrize("hospitalization_id,date_hr,expected_hr,expected_flag,expected_flag_noreintub", [
#     # on imv for 24-hrs twice during the same hospitalization -- so would be excluded if no reintubation within 72 hrs
#     (21738444, "2186-09-14 17:00:00-06:00", 24, 1, 0),  
#     (21738444, "2186-09-14 18:00:00-06:00", 24, 0, 0),  
#     (21738444, "2186-09-16 18:00:00-06:00", 24, 1, 0), # second streak within the hosp
#     # not on imv for the first few hrs but long streak afterwards
#     (20004088, "2159-09-30 09:00:00-06:00", 24, 1, 1),
#     (20004088, "2159-10-02 09:00:00-06:00", 72, 1, 1),
#     # very short streaks: 20006154
#     # 3 icu stays within the same hospitalization
#     (20018306, "2136-05-16 05:00:00-06:00", 24, 1, 1),
#     (20018306, "2136-07-01 19:00:00-06:00", 24, 1, 1),
#     # (20018306, "2136-06-01 03:00:00-06:00", 24, 0), # in a icu stay that was filtered out in the df because of no imv ever
# ])
# def test_if_on_imv_at_hr_x(hospitalization_id, date_hr, expected_hr, expected_flag, expected_flag_noreintub):
#     query = f"""
#     SELECT hr_{expected_hr}_on_imv, hr_{expected_hr}_on_imv_noreintub
#     FROM df2
#     WHERE hospitalization_id = {hospitalization_id}
#     AND date_hr = '{date_hr}'
#     """
#     result = duckdb.sql(query).to_df()
#     observed_flag = result[f'hr_{expected_hr}_on_imv'].iloc[0]
#     observed_flag_noreintub = result[f'hr_{expected_hr}_on_imv_noreintub'].iloc[0]
#     assert observed_flag == expected_flag
#     assert observed_flag_noreintub == expected_flag_noreintub

UsageError: Cell magic `%%ipytest` not found.


### The Cohort

In [69]:
# keep the cohort
query = """
SELECT ROW_NUMBER() OVER () as row_id
    , hospitalization_id
    --, encounter_block
    --, icu_stay_id
    , date_hr
    , CASE WHEN hr_24_on_imv_noreintub = 1 THEN 'hr_24'
        WHEN hr_72_on_imv_noreintub = 1 THEN 'hr_72'
        ELSE NULL END as cohort_flag
FROM df2
LEFT JOIN hosp_to_enc_blk_mapper USING (hospitalization_id)
WHERE hr_24_on_imv_noreintub = 1 OR hr_72_on_imv_noreintub = 1
"""
cohort = duckdb.sql(query).to_df()

cohort_hosp_ids = cohort['hospitalization_id'].unique().tolist()

## Demographics

In [44]:
hosp_required_columns = [
    "patient_id", 
    "hospitalization_id", 
    "age_at_admission"
]

cohort_hosp = pc.load_data(
    table = "clif_hospitalization",
    columns = hosp_required_columns,
    filters = {
        "hospitalization_id": cohort_hosp_ids
    }
)

cohort_pt_ids = cohort_hosp['patient_id'].unique().tolist()

pt_required_columns = [
    "patient_id",
    "race_category",
    "ethnicity_category",
    "sex_category"
]

cohort_pt = pc.load_data(
    table = "clif_patient",
    columns = pt_required_columns,
    filters = {
        "patient_id": cohort_pt_ids
    }
)

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_hospitalization.parquet
Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_patient.parquet


In [45]:
query = """
SELECT *
FROM cohort_hosp
LEFT JOIN cohort_pt
USING (patient_id)
"""
cohort_demogs = duckdb.sql(query).to_df()

## Vitals

In [17]:
vitals_required_columns = [
    "hospitalization_id",
    "recorded_dttm",
    "vital_category",
    "vital_value"
]

vitals = pc.load_data(
    table = "clif_vitals",
    filters = {
        "hospitalization_id": cohort_hosp_ids
    },
    columns = vitals_required_columns
)

vitals['date_hr'] = vitals['recorded_dttm'].dt.floor('h')

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_vitals.parquet


In [18]:
vitals = remove_outliers_with_timing(vitals, 'vitals', 'vital_value', file_path='config/outliers.json')

Outliers found in vitals
Category 'map': 3876 outliers (0.13%) have been set to NaN.
Category 'spo2': 1592 outliers (0.06%) have been set to NaN.
Category 'temp_c': 1104 outliers (0.14%) have been set to NaN.
Category 'respiratory_rate': 610 outliers (0.02%) have been set to NaN.
Category 'weight_kg': 499 outliers (0.26%) have been set to NaN.
Category 'dbp': 424 outliers (0.01%) have been set to NaN.
Category 'sbp': 86 outliers (0.00%) have been set to NaN.
Category 'height_cm': 32 outliers (0.29%) have been set to NaN.
Category 'heart_rate': 12 outliers (0.00%) have been set to NaN.
⏱️ Outlier removal completed in 7.12 seconds
🟢 Processed 18,115,764 rows


In [19]:
query = f"""
SELECT *
FROM cohort c
CROSS JOIN (SELECT DISTINCT vital_category FROM vitals) v
ORDER BY hospitalization_id, date_hr, vital_category
"""
cohort_hrs_cross_vital_categories = duckdb.sql(query).to_df()

In [20]:
query = """
-- fill any missing values in the cohort hours with the nearest in time (in the past or future)
SELECT hospitalization_id
    , date_hr
    , cohort_flag
    , vital_category
    , MEAN(vital_value) as mean_value
    
    , LAG(mean_value) OVER (PARTITION BY hospitalization_id, vital_category ORDER BY date_hr) as mean_value_lag
    , LAST_VALUE(mean_value IGNORE NULLS) OVER (
        PARTITION BY hospitalization_id, vital_category 
        ORDER BY date_hr
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) as mean_value_final
    -- , COALESCE(mean_value, mean_value_last) as mean_value_final
FROM cohort_hrs_cross_vital_categories
FULL OUTER JOIN vitals USING (hospitalization_id, date_hr, vital_category)
GROUP BY hospitalization_id, date_hr, cohort_flag, vital_category
ORDER BY hospitalization_id, vital_category, date_hr, cohort_flag
"""
vitals_hrly = duckdb.sql(query).to_df()

In [21]:
query = """
SELECT hospitalization_id
    , date_hr
    , cohort_flag
    , vital_category
    , mean_value
    , mean_value_final
FROM vitals_hrly
WHERE cohort_flag IS NOT NULL
ORDER BY hospitalization_id, vital_category, date_hr, cohort_flag
"""
vitals_cohort_hrs = duckdb.sql(query).to_df()

In [51]:
vitals_cohort_hrs_w = vitals_cohort_hrs.pivot_table(
    index=['hospitalization_id', 'date_hr', 'cohort_flag'], 
    columns='vital_category', 
    values='mean_value_final', 
    fill_value=0
).reset_index()
vitals_cohort_hrs_w.columns.name = None

## Assessments

In [102]:
pa_required_columns = [
    "hospitalization_id", 
    "recorded_dttm",
    "assessment_category", 
    "numerical_value"
]

pa = pc.load_data(
    table = "clif_patient_assessments",
    columns = pa_required_columns,
    filters = {
        "hospitalization_id": cohort_hosp_ids,
        "assessment_category": ["gcs_total", "rass", "RASS"]
    }
)

pa['date_hr'] = pa['recorded_dttm'].dt.floor('h')

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_patient_assessments.parquet


In [ ]:
# pa_required_columns = [
#     "hospitalization_id", 
#     "recorded_dttm",
#     "assessment_category", 
#     "numerical_value"
# ]

# gcs = pc.load_data(
#     table = "clif_patient_assessments",
#     columns = pa_required_columns,
#     filters = {
#         "hospitalization_id": cohort_hosp_ids,
#         "assessment_category": ["gcs_total", #"rass", "RASS", 
#                                 "gcs_verbal", "gcs_motor", "gcs_eye"]
#     }
# )

# gcs['date_hr'] = gcs['recorded_dttm'].dt.floor('h')

# gcs_w = gcs.pivot_table(
#     index=['hospitalization_id', 'recorded_dttm'],
#     columns='assessment_category',
#     values='numerical_value',
#     aggfunc='mean'
# ).reset_index()

# # Flatten column names
# gcs_w.columns.name = None

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_patient_assessments.parquet


### Intubaton start hour

In [78]:
query = """
SELECT *
    -- note that we subtract 23 instead of 24 to be inclusive of all minutes in the starting hr
    , CASE WHEN cohort_flag = 'hr_24' THEN date_hr - INTERVAL '20 hours' 
        WHEN cohort_flag = 'hr_72' THEN date_hr - INTERVAL '68 hours'
        ELSE NULL END as date_hr_intub
FROM cohort c
"""
cohort_intub = duckdb.sql(query).to_df()

### Most recent GCS

In [98]:
def find_most_recent_gcs(target_dttm_name: str = 'date_hr_intub', window_in_hr: int = 22):
    '''
    Find the most recent GCS assessment to the target time.
    '''
    query = f"""
    WITH t1 AS (
        SELECT *
            , CASE WHEN cohort_flag = 'hr_24' THEN date_hr - INTERVAL '{window_in_hr} hours' 
                WHEN cohort_flag = 'hr_72' THEN date_hr - INTERVAL '{window_in_hr + 48} hours'
                ELSE NULL END as date_hr_intub
        FROM cohort c
    )
    SELECT c.*
        , p.numerical_value as gcs_total
        , p.recorded_dttm as gcs_recorded_dttm
        -- rn = 1 for the gcs w/ the latest recorded_dttm (and thus most recent)
        , ROW_NUMBER() OVER (
            PARTITION BY c.row_id, c.hospitalization_id, c.{target_dttm_name}
            ORDER BY p.recorded_dttm DESC
            ) as rn
    FROM t1 c
    LEFT JOIN pa p
        ON c.hospitalization_id = p.hospitalization_id 
        AND p.assessment_category = 'gcs_total'
        AND p.recorded_dttm <= c.{target_dttm_name}
    QUALIFY (rn = 1) OR (gcs_total IS NULL) -- include cohort even if no gcs found
    ORDER BY row_id, c.{target_dttm_name}, p.recorded_dttm
    """
    return duckdb.sql(query).to_df()

pa_gcs_intub_hr = find_most_recent_gcs(target_dttm_name='date_hr_intub')
pa_gcs_cohort_hr = find_most_recent_gcs(target_dttm_name='date_hr')


### Most recent RASS

In [ ]:
def find_most_recent(clif_category: str, hr_from_cohort_dttm: int = 1):
    '''
    Find the most recent assessment to the target time.
    '''
    query = f"""
    WITH t1 AS (
        SELECT *
            , CASE WHEN cohort_flag = 'hr_24' THEN date_hr - INTERVAL '{hr_from_cohort_dttm} hours' 
                WHEN cohort_flag = 'hr_72' THEN date_hr - INTERVAL '{hr_from_cohort_dttm + 48} hours'
                ELSE NULL END as date_hr_intub
            , date_hr + INTERVAL '{hr_from_cohort_dttm} hours' as target_dttm
        FROM cohort c
    )
    SELECT c.*
        , p.numerical_value as value
        , p.recorded_dttm
        -- rn = 1 for the gcs w/ the latest recorded_dttm (and thus most recent)
        , ROW_NUMBER() OVER (
            PARTITION BY c.row_id, c.hospitalization_id, c.target_dttm
            ORDER BY p.recorded_dttm DESC
            ) as rn
    FROM t1 c
    LEFT JOIN pa p
        ON c.hospitalization_id = p.hospitalization_id 
        AND p.assessment_category in ('{clif_category}', '{clif_category.lower()}', '{clif_category.upper()}')
        AND p.recorded_dttm <= c.target_dttm
    QUALIFY (rn = 1) OR (value IS NULL) -- include cohort even if no clif_category found
    ORDER BY row_id, c.target_dttm, p.recorded_dttm
    """
    return duckdb.sql(query).to_df()

cohort_rass = find_most_recent(clif_category='rass', hr_from_cohort_dttm=1)


## Medication

In [258]:
sed_med_categories = [
    "midazolam", "lorazepam", "hydromorphone", "fentanyl", "propofol", "dexmedetomidine", "ketamine"
]

vaso_med_categories = [
    "epinephrine", "norepinephrine", "phenylephrine", "vasopressin", "angiotensin", 
    "dopamine", "dobutamine"
]

mac_required_columns = [
    "hospitalization_id", 
    "admin_dttm",
    "med_category",
    "med_group",
    "med_dose",
    "med_dose_unit",
    "mar_action_name"
]

mac = pc.load_data(
    table = "clif_medication_admin_continuous",
    columns = mac_required_columns,
    filters = {
        "hospitalization_id": cohort_hosp_ids,
        "med_category": sed_med_categories + vaso_med_categories
    }
)

mac['date_hr'] = mac['admin_dttm'].dt.floor('h')

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_medication_admin_continuous.parquet


### Check dosage unit

In [259]:
query = """
SELECT med_category, med_dose_unit, COUNT(*) as n
FROM mac
GROUP BY med_category, med_dose_unit
ORDER BY med_category, n DESC
"""
med_units_count = duckdb.sql(query).to_df()
med_units_count

,med_category,med_dose_unit,n
0,angiotensin,ng/kg/min,221
1,angiotensin,mcg/kg/min,103
2,dexmedetomidine,mcg/kg/hour,89430
3,dobutamine,mcg/kg/min,5238
4,dopamine,mcg/kg/min,8808
5,epinephrine,mcg/kg/min,19074
6,fentanyl,mcg/hour,113807
7,fentanyl,mcg/kg/hour,2
8,hydromorphone,mg/hour,9356
9,ketamine,mg/kg/hour,7513


### Most recent patient weight

In [260]:
mac.columns

Index(['hospitalization_id', 'admin_dttm', 'med_category', 'med_group',
       'med_dose', 'med_dose_unit', 'mar_action_name', 'date_hr'],
      dtype='object')

In [261]:
query = """
SELECT m.*
    , v.vital_value as weight_kg
    , v.recorded_dttm as weight_recorded_dttm
    -- rn = 1 for the weight w/ the latest recorded_dttm (and thus most recent)
    , ROW_NUMBER() OVER (
        PARTITION BY m.hospitalization_id, m.admin_dttm, m.med_category
        ORDER BY v.recorded_dttm DESC
        ) as rn
FROM mac m
LEFT JOIN vitals v 
    ON m.hospitalization_id = v.hospitalization_id 
    AND v.vital_category = 'weight_kg'
    AND v.recorded_dttm <= m.admin_dttm  -- only past weights
QUALIFY (rn = 1) OR (weight_kg IS NULL) -- include meds even if no weight found
ORDER BY m.hospitalization_id, m.admin_dttm, m.med_category, rn
"""
mac_w_wt = duckdb.sql(query).to_df()

### Standardize dosage unit

In [262]:
def standardize_dose_unit(df_name: str) -> pd.DataFrame:
    """
    Standardize everything to mcg/min.
    Assumes the presentation of the following columns:
    - med_dose_unit: the original unit of the dose
    - med_dose: the original dose
    - weight_kg: the (imputed, most recent) weight of the patient
    """
    query = f"""
    SELECT *
        , LOWER(med_dose_unit) AS med_dose_unit_lower
        , CASE WHEN regexp_matches(med_dose_unit_lower, '/h(r|our)?\\b') THEN 1/60.0
            WHEN regexp_matches(med_dose_unit_lower, '/m(in|inute)?\\b') THEN 1.0
            ELSE NULL END as time_multiplier
        , CASE WHEN contains(med_dose_unit_lower, '/kg/') THEN weight_kg
            ELSE 1 END AS pt_weight_adjustment
        , CASE WHEN contains(med_dose_unit_lower, 'mcg/') THEN 1.0
            WHEN contains(med_dose_unit_lower, 'mg/') THEN 1000.0
            WHEN contains(med_dose_unit_lower, 'ng/') THEN 0.001
            WHEN contains(med_dose_unit_lower, 'milli') THEN 0.001
            ELSE NULL END as dose_mass_multiplier
        , med_dose * time_multiplier * pt_weight_adjustment * dose_mass_multiplier as med_dose_converted
        , CASE WHEN contains(med_dose_unit_lower, 'units/') THEN 'units/min'
            ELSE 'mcg/min' END as med_dose_unit_converted
    FROM {df_name}
    """
    return duckdb.sql(query).to_df()

mac_converted = standardize_dose_unit('mac_w_wt')

### Scaffold cohort with med_categories

In [263]:
query = f"""
SELECT *
FROM cohort c
CROSS JOIN (SELECT DISTINCT med_category FROM mac) m
ORDER BY hospitalization_id, date_hr, med_category
"""
cohort_hrs_cross_med_categories = duckdb.sql(query).to_df()

### Calculate cumulative dosage

In [264]:
query = """
-- insert the cohort hours into the mac record
SELECT hospitalization_id
    , cohort_flag
    , med_category
    , date_hr
    , admin_dttm
    , mar_action_name
    -- , med_dose, med_dose_unit
    , med_dose_converted as med_dose
    , med_dose_unit_converted as med_dose_unit
    , weight_kg
    , LAST_VALUE(admin_dttm IGNORE NULLS) OVER (
        PARTITION BY hospitalization_id, med_category 
        ORDER BY date_hr, admin_dttm 
        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ) as admin_dttm_last
    , LAST_VALUE(med_dose IGNORE NULLS) OVER (
        PARTITION BY hospitalization_id, med_category 
        ORDER BY date_hr, admin_dttm 
        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ) as med_dose_last
    , LAST_VALUE(mar_action_name IGNORE NULLS) OVER (
        PARTITION BY hospitalization_id, med_category 
        ORDER BY date_hr, admin_dttm 
        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ) as mar_action_name_last
    -- add helper flags for first and last streak of the same med within the hour -- which needs special handling
    , CASE WHEN admin_dttm = MIN(admin_dttm) OVER (PARTITION BY hospitalization_id, med_category, date_hr) 
        THEN 1 ELSE 0 END as is_first_streak
    , CASE WHEN admin_dttm = MAX(admin_dttm) OVER (PARTITION BY hospitalization_id, med_category, date_hr) 
        THEN 1 ELSE 0 END as is_last_streak
    , date_hr + INTERVAL '1 hour' as date_hr_next
FROM cohort_hrs_cross_med_categories c
FULL OUTER JOIN mac_converted m USING (hospitalization_id, date_hr, med_category)
ORDER BY hospitalization_id, med_category, date_hr, admin_dttm
"""
mac_hrly = duckdb.sql(query).to_df()

In [265]:
# query = """
# SELECT *

# FROM mac_hrly
# ORDER BY hospitalization_id, med_category, date_hr, admin_dttm
# """
# # forward filled
# mac_hrly_ff = duckdb.sql(query).to_df()

In [266]:
query = """
-- keep only the med admins that are within the cohort hours
SELECT *
FROM mac_hrly
WHERE cohort_flag IS NOT NULL -- AND admin_dttm_last IS NOT NULL
ORDER BY hospitalization_id, med_category, date_hr, admin_dttm
"""
mac_cohort_hrs = duckdb.sql(query).to_df()

In [267]:
mask = mac_cohort_hrs['cohort_flag'] == 'hr_24'
mask.sum()

np.int64(211849)

In [274]:
query = f"""
-- calculate the cumulative dosage within the hr
SELECT hospitalization_id, cohort_flag, med_category, date_hr
    , SUM(CASE 
        -- if no mac admin record within the cohort hour, use the last observed dose and assume it runs the entire 60 mins
        WHEN admin_dttm IS NULL
            THEN 60.0 * COALESCE(med_dose_last, 0)
        -- otherwise, calculate the cumulative dosage within the hr
        WHEN is_first_streak = 1
            THEN EXTRACT(EPOCH FROM (admin_dttm - date_hr))/60.0 * med_dose_last 
        WHEN is_first_streak != 1 AND admin_dttm IS NOT NULL
            THEN EXTRACT(EPOCH FROM (admin_dttm - admin_dttm_last))/60.0 * med_dose_last 
        WHEN is_last_streak = 1
            THEN EXTRACT(EPOCH FROM (date_hr_next - admin_dttm))/60.0 * med_dose
        ELSE 0 END) as total_dosage
    , COUNT(DISTINCT CASE 
        WHEN total_dosage > 0 AND med_category in {vaso_med_categories}
        THEN med_category END) OVER (PARTITION BY hospitalization_id, date_hr) as n_pressors
FROM mac_cohort_hrs
GROUP BY hospitalization_id, med_category, date_hr, cohort_flag, med_dose_unit
ORDER BY hospitalization_id, med_category, date_hr
"""
mac_cohort_dosage = duckdb.sql(query).to_df()

In [276]:
mac_cohort_dosage_w = mac_cohort_dosage.pivot_table(
    index=['hospitalization_id', 'date_hr', 'cohort_flag', 'n_pressors'], 
    columns='med_category', 
    values='total_dosage', 
    fill_value=0
).reset_index()
mac_cohort_dosage_w.columns.name = None

In [270]:
# query = """
# SELECT hospitalization_id
#     , date_hr
#     , cohort_flag
#     , med_category
#     , COALESCE(total_dosage, 0) as total_dosage
#     , med_dose_unit
# FROM cohort_hrs_cross_med_categories c
# FULL OUTER JOIN mac_cohort_dosage m USING (hospitalization_id, date_hr, med_category, cohort_flag)
# ORDER BY hospitalization_id, cohort_flag, med_category, date_hr
# """
# df = duckdb.sql(query).to_df()

In [271]:
mac_cohort_dosage['med_category'].unique().tolist()

['angiotensin',
 'dexmedetomidine',
 'dobutamine',
 'dopamine',
 'epinephrine',
 'fentanyl',
 'hydromorphone',
 'ketamine',
 'lorazepam',
 'midazolam',
 'norepinephrine',
 'phenylephrine',
 'propofol',
 'vasopressin']

## SOFA score

In [38]:
query = """
SELECT row_id
    , date_hr - INTERVAL '23 hours' as start_dttm
    , date_hr + INTERVAL '1 hours'as stop_dttm
FROM cohort
"""
sofa_input = duckdb.sql(query).to_df()

In [39]:
query = """
SELECT row_id
    , hospitalization_id
FROM cohort
"""
id_mappings = duckdb.sql(query).to_df()

In [ ]:
from utils import sofa_score
reload(sofa_score)

Loaded configuration from config.json
Loaded configuration from config.json


In [41]:
sofa_score = sofa_score.compute_sofa(
    ids_w_dttm=sofa_input,
    tables_path=helper['tables_path'],
    use_hospitalization_id=False,
    id_mapping=id_mappings,
    output_filepath="output/intermediate/sofa.parquet",
    helper_module=pc
)

2025-08-05 16:18:39,784 - INFO - Starting SOFA computation for 20244 rows
2025-08-05 16:18:40,281 - INFO - Loaded 942691 lab rows


Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_labs.parquet
lab_collect_dttm: null count before conversion= 0
lab_collect_dttm: Your timezone is UTC, Converting to your site timezone (US/Eastern).
lab_collect_dttm: null count after conversion= 0


2025-08-05 16:18:42,202 - INFO - Loaded 5869871 vitals rows


Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_vitals.parquet
recorded_dttm: null count before conversion= 0
recorded_dttm: Your timezone is UTC, Converting to your site timezone (US/Eastern).
recorded_dttm: null count after conversion= 0


2025-08-05 16:18:44,449 - INFO - Created vitals summary
2025-08-05 16:18:44,452 - INFO - Imputed pao2 from spo2
2025-08-05 16:18:44,617 - INFO - Loaded 674017 GCS rows


...

Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_medication_admin_continuous.parquet
admin_dttm: null count before conversion= 0
admin_dttm: Your timezone is UTC, Converting to your site timezone (US/Eastern).
admin_dttm: null count after conversion= 0


2025-08-05 16:18:47,097 - INFO - Loaded 845617 resp rows


Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_respiratory_support.parquet
recorded_dttm: null count before conversion= 0
recorded_dttm: Your timezone is UTC, Converting to your site timezone (US/Eastern).
recorded_dttm: null count after conversion= 0
FIO2_SET mean= 0.5049472965993077 is within the required range
Missing ratio of p_f (po2_arterial_recent / fio2_recent):  0.10896483407627539
Missing ratio of p_f_imputed (pao2_imputed_recent / fio2_recent):  0.7003467062902426
Missing ratio of s_f (spo2_recent / fio2_recent): 0.0005448241703813769

Most of the missing values in p_f_imputed are caused by pao2_imputed_recent, which is set to NaN when spo2>97
Data loaded successfully from /Users/wliao0504/code/clif/CLIF-MIMIC/output/rclif-dev-test-copy/clif_crrt_therapy.parquet


2025-08-05 16:18:50,455 - INFO - Loaded 467609 CRRT rows
2025-08-05 16:18:50,593 - INFO - Finished computing SOFA for 20190 ids
2025-08-05 16:18:50,607 - INFO - Wrote results to output/intermediate/sofa.parquet


recorded_dttm: null count before conversion= 0
recorded_dttm: Your timezone is UTC, Converting to your site timezone (US/Eastern).
recorded_dttm: null count after conversion= 0


In [42]:
sofa_score.columns

Index(['row_id', 'fio2_recent', 'resp_support_recent', 'map_recent',
       'spo2_recent', 'pao2_imputed_recent', 'bilirubin_total_recent',
       'creatinine_recent', 'platelet_count_recent', 'po2_arterial_recent',
       'gcs_score_recent', 'angiotensin', 'dobutamine', 'dopamine',
       'epinephrine', 'norepinephrine', 'phenylephrine', 'vasopressin', 'p_f',
       'p_f_imputed', 's_f', 'sofa_cv_97', 'sofa_coag', 'sofa_liver',
       'sofa_resp_pf', 'sofa_resp_pf_imp', 'sofa_resp', 'sofa_cns',
       'sofa_renal', 'crrt_flag', 'sofa_total'],
      dtype='object')

In [301]:
sofa_score.drop(columns=vaso_med_categories, inplace=True)

## Merge results

In [272]:
vaso_med_categories

['epinephrine',
 'norepinephrine',
 'phenylephrine',
 'vasopressin',
 'angiotensin',
 'dopamine',
 'dobutamine']

In [ ]:
ADULT_WEIGHT_PROXY = 70

query = f"""
SELECT row_id
    , hospitalization_id, date_hr, cohort_flag
    , EXTRACT(hour FROM date_hr) as hr
    , CASE 
        WHEN EXTRACT(hour FROM date_hr) >= 7 AND EXTRACT(hour FROM date_hr) < 19 
        THEN 'day' ELSE 'night' END AS shift
    , d.age_at_admission as age
    , d.race_category as race
    , d.ethnicity_category as ethnicity
    , d.sex_category as sex
    , v.*
    , r.value as rass
    -- sedatives
    , LEAST(m.midazolam, 60*1000) as midazolam
    , LEAST(m.lorazepam, 10*1000) as lorazepam
    , LEAST(m.hydromorphone, 3*1000) as hydromorphone
    , LEAST(m.fentanyl, 700) as fentanyl -- too low?
    , LEAST(m.propofol, 4000*v.weight_kg) as propofol
    , LEAST(m.dexmedetomidine, 1.5*v.weight_kg) as dexmedetomidine
    , LEAST(m.ketamine, 4000*v.weight_kg) as ketamine
    -- vasopressors
    , LEAST(m.norepinephrine, 3*60*v.weight_kg) as norepinephrine
    , LEAST(m.epinephrine, 0.1*60*v.weight_kg) as epinephrine
    , LEAST(m.phenylephrine, 5*60*v.weight_kg) as phenylephrine
    , LEAST(m.dopamine, 20*60*v.weight_kg) as dopamine
    , LEAST(m.dobutamine, 40*60*v.weight_kg) as dobutamine
    , LEAST(m.vasopressin, 0.1*60*v.weight_kg) as vasopressin
    , LEAST(m.angiotensin, 0.2*60*v.weight_kg) as angiotensin
    -- converted equivalents
    , hydromorphone * 0.05 + fentanyl AS fentanyl_eq
    , lorazepam * 2 + m.midazolam AS midazolam_eq
    , norepinephrine + epinephrine + phenylephrine / 10.0 + dopamine / 100.0 
        + vasopressin * 2.5 + angiotensin * 10 as ne_eq
    , n_pressors
    , s.sofa_total as sofa
    , COALESCE(s.po2_arterial_recent, s.pao2_imputed_recent) AS pao2
    , s.fio2_recent * 100 AS fio2
    , COALESCE(s.p_f, s.p_f_imputed) / 100 AS p_f
FROM cohort c
LEFT JOIN cohort_demogs d USING (hospitalization_id)
LEFT JOIN vitals_cohort_hrs_w v USING (hospitalization_id, date_hr, cohort_flag)
LEFT JOIN cohort_rass r USING (row_id)
LEFT JOIN mac_cohort_dosage_w m USING (hospitalization_id, date_hr, cohort_flag)
LEFT JOIN sofa_score s USING (row_id) 
ORDER BY row_id
"""
cohort_results = duckdb.sql(query).to_df()

cohort_results_24 = cohort_results[cohort_results['cohort_flag'] == 'hr_24'].copy()

cohort_results_72 = cohort_results[cohort_results['cohort_flag'] == 'hr_72'].copy()

## Table one

In [325]:
cohort_results.columns

Index(['row_id', 'hospitalization_id', 'date_hr', 'cohort_flag', 'hr', 'shift',
       'age', 'race', 'ethnicity', 'sex', 'hospitalization_id_1', 'date_hr_1',
       'cohort_flag_1', 'dbp', 'heart_rate', 'height_cm', 'map',
       'respiratory_rate', 'sbp', 'spo2', 'temp_c', 'weight_kg', 'rass',
       'midazolam', 'lorazepam', 'hydromorphone', 'fentanyl', 'propofol',
       'dexmedetomidine', 'ketamine', 'norepinephrine', 'epinephrine',
       'phenylephrine', 'dopamine', 'dobutamine', 'vasopressin', 'angiotensin',
       'fentanyl_eq', 'midazolam_eq', 'ne_eq', 'n_pressors', 'sofa', 'pao2',
       'fio2', 'p_f'],
      dtype='object')

In [326]:
vitals_vars = [
    'sbp', 'dbp', 'map', 'heart_rate', 'respiratory_rate', 'spo2', 'temp_c', 'weight_kg', 'height_cm'
    ]

resp_vars = ['pao2', 'fio2', 'p_f']

cat_vars = ['race', 'ethnicity', 'sex']
cont_vars = ['age', 'rass', 'sofa', 'fentanyl_eq', 'midazolam_eq', 'ne_eq', 'n_pressors'] \
    + resp_vars + vitals_vars + sed_med_categories + vaso_med_categories


In [327]:
def gen_and_save_tableone(file_name, **kwargs):
    """
    Wrapper for tableone.TableOne that automatically saves results to a CSV file.
    
    Args:
        data: DataFrame to create table one from
        file_name: Name for the output file (without extension)
        **kwargs: All other arguments passed to tableone.TableOne
    
    Returns:
        tableone.TableOne object
    """
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    table = tableone.TableOne(**kwargs, pval=True)
    table.to_csv(f'output/final/{site}_{file_name}_{timestamp}.csv')
    return table


### By cohort

In [328]:
gen_and_save_tableone(
    file_name='table_one_by_cohort_hr',
    data=cohort_results, 
    continuous=cont_vars, 
    categorical=cat_vars, 
    groupby='cohort_flag',
    nonnormal=['age', 'rass', 'sofa'] + resp_vars + vitals_vars
    )

Grouped by cohort_flag                                                                       
                                                                                          Missing              Overall                hr_24                hr_72 P-Value
n                                                                                                                20244                14957                 5287        
age, median [Q1,Q3]                                                                             0     65.0 [54.0,75.0]     65.0 [54.0,76.0]     65.0 [53.0,75.0]   0.012
race, n (%)                      American Indian or Alaska Native                                             52 (0.3)             41 (0.3)             11 (0.2)  <0.001
                                 Asian                                                                       569 (2.8)            421 (2.8)            148 (2.8)        
                                 Black or African American                                                 2136 (10.6)          1581 (10.6)           555 (10.5)        
                                 Native Hawaiian or Other Pacific Islander                                    38 (0.2)             29 (0.2)              9 (0.2)        
                                 Other                                                                      1374 (6.8)           1009 (6.7)            365 (6.9)        
                                 Unknown                                                                   3882 (19.2)          2731 (18.3)          1151 (21.8)        
                                 White                                                                    12193 (60.2)          9145 (61.1)          3048 (57.7)        
ethnicity, n (%)                 Hispanic                                                                    757 (3.7)            562 (3.8)            195 (3.7)  <0.001
                                 Non-Hispanic                                                             14988 (74.0)         11217 (75.0)          3771 (71.3)        
                                 Unknown                                                                   4499 (22.2)          3178 (21.2)          1321 (25.0)        
sex, n (%)                       Female                                                                    8541 (42.2)          6310 (42.2)          2231 (42.2)   1.000
                                 Male                                                                     11703 (57.8)          8647 (57.8)          3056 (57.8)        
dbp, median [Q1,Q3]                                                                             2     58.0 [51.0,67.0]     58.0 [51.0,67.0]     59.0 [51.8,68.0]   0.001
heart_rate, median [Q1,Q3]                                                                      2     84.0 [72.0,97.0]     84.0 [71.0,98.0]     84.0 [72.0,97.0]   0.831
height_cm, median [Q1,Q3]                                                                       2  168.0 [157.0,175.0]  168.0 [155.0,175.0]  168.0 [157.0,178.0]  <0.001
map, median [Q1,Q3]                                                                             2     75.0 [68.0,85.0]     75.0 [67.5,84.0]     76.0 [69.0,87.0]  <0.001
respiratory_rate, median [Q1,Q3]                                                                2     19.0 [16.0,24.0]     19.0 [16.0,23.5]     20.0 [16.0,24.0]  <0.001
sbp, median [Q1,Q3]                                                                             2  114.0 [102.0,129.0]  113.0 [101.6,128.0]  117.0 [104.0,133.0]  <0.001
spo2, median [Q1,Q3]                                                                            2    98.0 [96.0,100.0]    98.0 [96.0,100.0]     98.0 [96.0,99.5]  <0.001
temp_c, median [Q1,Q3]                                                                          2     37.1 [36.7,37.5]     37.1 [36.7,37.5]     37.1 [36.7,37.6]  <0.001
weight_kg, median [

### By race

In [329]:
gen_and_save_tableone(
    file_name='table_one_by_race',
    data=cohort_results, 
    continuous=cont_vars, 
    categorical=['ethnicity', 'sex'], 
    groupby='race',
    nonnormal=['age', 'rass', 'sofa'] + resp_vars + vitals_vars
    )

Grouped by race                                                                                                                                                                                                                      
                                                      Missing              Overall American Indian or Alaska Native                Asian Black or African American Native Hawaiian or Other Pacific Islander                Other              Unknown                White P-Value
n                                                                            20244                               52                  569                      2136                                        38                 1374                 3882                12193        
age, median [Q1,Q3]                                         0     65.0 [54.0,75.0]                 63.5 [58.0,73.8]     67.0 [56.0,81.0]          65.0 [55.0,75.0]                          65.5 [52.5,78.0]     59.0 [46.0,71.0]     63.0 [49.0,74.0]     66.0 [55.0,76.0]  <0.001
ethnicity, n (%)                 Hispanic                                757 (3.7)                          0 (0.0)              0 (0.0)                   0 (0.0)                                   0 (0.0)           757 (55.1)              0 (0.0)              0 (0.0)  <0.001
                                 Non-Hispanic                         14988 (74.0)                       52 (100.0)          569 (100.0)              2136 (100.0)                                38 (100.0)              0 (0.0)              0 (0.0)        12193 (100.0)        
                                 Unknown                               4499 (22.2)                          0 (0.0)              0 (0.0)                   0 (0.0)                                   0 (0.0)           617 (44.9)         3882 (100.0)              0 (0.0)        
sex, n (%)                       Female                                8541 (42.2)                        27 (51.9)           239 (42.0)               1144 (53.6)                                 12 (31.6)           554 (40.3)          1494 (38.5)          5071 (41.6)  <0.001
                                 Male                                 11703 (57.8)                        25 (48.1)           330 (58.0)                992 (46.4)                                 26 (68.4)           820 (59.7)          2388 (61.5)          7122 (58.4)        
dbp, median [Q1,Q3]                                         2     58.0 [51.0,67.0]                 55.0 [47.0,63.1]     58.0 [52.0,67.0]          61.0 [54.0,71.0]                          62.5 [58.0,69.8]     59.0 [53.0,68.0]     60.0 [52.0,69.0]     57.0 [50.0,66.0]  <0.001
heart_rate, median [Q1,Q3]                                  2     84.0 [72.0,97.0]                 85.4 [73.5,96.2]    84.0 [72.0,101.0]          85.0 [72.0,98.0]                          83.0 [65.8,99.0]     85.0 [72.0,99.0]     84.0 [72.0,99.0]     83.0 [71.0,97.0]  <0.001
height_cm, median [Q1,Q3]                                   2  168.0 [157.0,175.0]              165.0 [156.5,173.0]  157.0 [152.0,168.0]       165.0 [155.0,175.0]                       171.0 [160.0,178.0]  165.0 [152.0,173.0]  168.0 [157.0,176.5]  168.0 [157.0,178.0]  <0.001
map, median [Q1,Q3]                                         2     75.0 [68.0,85.0]                 72.0 [63.8,82.8]     75.0 [67.5,84.0]          77.0 [69.0,88.0]                          80.0 [73.2,88.2]     76.0 [69.0,86.4]     76.0 [68.5,86.0]     74.0 [67.0,83.3]  <0.001
respiratory_rate, median [Q1,Q3]                            2     19.0 [16.0,24.0]                 18.7 [16.0,23.2]     19.0 [15.0,24.0]          19.0 [16.0,23.5]                          20.0 [16.0,23.8]     19.0 [16.0,24.0]     20.0 [16.0,24.0]     19.0 [15.5,23.0]  <0.001
sbp, median [Q1,Q3]                                         2  114.0 [102.0,129.0]              118.5 [101.0,133.2]  113.0 [100.0,128.0]       116.0 [103.0,133.0]                    

### By shift

In [330]:
gen_and_save_tableone(
    file_name='table_one_by_shift_hr24',
    data=cohort_results_24, 
    continuous=cont_vars, 
    categorical=['race', 'ethnicity', 'sex'], 
    groupby='shift'
    # nonnormal=['age', 'rass', 'sofa'] + resp_vars + vitals_vars,    
    )

Grouped by shift                                                             
                                                                               Missing           Overall              day             night P-Value
n                                                                                                  14957             8122              6835        
age, mean (SD)                                                                       0       63.6 (16.3)      64.2 (15.8)       62.8 (16.8)  <0.001
race, n (%)                 American Indian or Alaska Native                                    41 (0.3)         25 (0.3)          16 (0.2)  <0.001
                            Asian                                                              421 (2.8)        234 (2.9)         187 (2.7)        
                            Black or African American                                        1581 (10.6)       857 (10.6)        724 (10.6)        
                            Native Hawaiian or Other Pacific Islander                           29 (0.2)         16 (0.2)          13 (0.2)        
                            Other                                                             1009 (6.7)        536 (6.6)         473 (6.9)        
                            Unknown                                                          2731 (18.3)      1352 (16.6)       1379 (20.2)        
                            White                                                            9145 (61.1)      5102 (62.8)       4043 (59.2)        
ethnicity, n (%)            Hispanic                                                           562 (3.8)        307 (3.8)         255 (3.7)  <0.001
                            Non-Hispanic                                                    11217 (75.0)      6234 (76.8)       4983 (72.9)        
                            Unknown                                                          3178 (21.2)      1581 (19.5)       1597 (23.4)        
sex, n (%)                  Female                                                           6310 (42.2)      3463 (42.6)       2847 (41.7)   0.231
                            Male                                                             8647 (57.8)      4659 (57.4)       3988 (58.3)        
dbp, mean (SD)                                                                       1       59.8 (12.8)      59.5 (12.8)       60.1 (12.9)   0.007
heart_rate, mean (SD)                                                                1       85.3 (19.0)      85.5 (18.8)       85.0 (19.2)   0.081
height_cm, mean (SD)                                                                 1      142.8 (62.1)     144.1 (60.8)      141.3 (63.6)   0.008
map, mean (SD)                                                                       1       76.8 (13.6)      76.6 (13.6)       76.9 (13.5)   0.267
respiratory_rate, mean (SD)                                                          1        19.6 (6.1)       19.6 (6.1)        19.5 (6.1)   0.347
sbp, mean (SD)                                                                       1      115.7 (20.2)     115.7 (20.4)      115.8 (19.9)   0.791
spo2, mean (SD)                                                                      1        97.5 (3.4)       97.4 (3.4)        97.6 (3.4)   0.005
temp_c, mean (SD)                                                                    1        35.5 (7.6)       35.1 (8.3)        35.9 (6.6)  <0.001
weight_kg, mean (SD)                                                                 1       85.2 (26.1)      85.8 (26.6)       84.6 (25.5)   0.006
rass, mean (SD)                                                                   1363        -2.4 (1.8)       -2.4 (1.8)        -2.4 (1.9)   0.089
midazolam, mean (SD)                                                                 0      31.7 (107.2)      28.9 (98.1)      35.0 (117.1)  <0.001
lorazepam, mean (SD)                                                      

In [331]:
gen_and_save_tableone(
    file_name='table_one_by_shift_hr72',
    data=cohort_results_72, 
    continuous=cont_vars, 
    categorical=['race', 'ethnicity', 'sex'], 
    groupby='shift',
    # nonnormal=['age', 'rass', 'sofa'] + resp_vars + vitals_vars,
    )

Grouped by shift                                                           
                                                                               Missing          Overall              day            night P-Value
n                                                                                                  5287             2821             2466        
age, mean (SD)                                                                       0      63.0 (16.3)      63.5 (15.9)      62.4 (16.8)   0.013
race, n (%)                 American Indian or Alaska Native                                   11 (0.2)          6 (0.2)          5 (0.2)   0.110
                            Asian                                                             148 (2.8)         80 (2.8)         68 (2.8)        
                            Black or African American                                        555 (10.5)       304 (10.8)       251 (10.2)        
                            Native Hawaiian or Other Pacific Islander                           9 (0.2)          5 (0.2)          4 (0.2)        
                            Other                                                             365 (6.9)        173 (6.1)        192 (7.8)        
                            Unknown                                                         1151 (21.8)       587 (20.8)       564 (22.9)        
                            White                                                           3048 (57.7)      1666 (59.1)      1382 (56.0)        
ethnicity, n (%)            Hispanic                                                          195 (3.7)         98 (3.5)         97 (3.9)   0.012
                            Non-Hispanic                                                    3771 (71.3)      2061 (73.1)      1710 (69.3)        
                            Unknown                                                         1321 (25.0)       662 (23.5)       659 (26.7)        
sex, n (%)                  Female                                                          2231 (42.2)      1224 (43.4)      1007 (40.8)   0.065
                            Male                                                            3056 (57.8)      1597 (56.6)      1459 (59.2)        
dbp, mean (SD)                                                                       1      60.5 (13.0)      60.4 (13.0)      60.5 (13.1)   0.965
heart_rate, mean (SD)                                                                1      85.2 (18.2)      86.0 (18.4)      84.2 (17.8)  <0.001
height_cm, mean (SD)                                                                 1     152.2 (51.8)     152.1 (51.6)     152.3 (51.9)   0.880
map, mean (SD)                                                                       1      78.6 (14.6)      78.7 (14.8)      78.5 (14.4)   0.666
respiratory_rate, mean (SD)                                                          1       19.9 (6.3)       20.0 (6.3)       19.9 (6.2)   0.543
sbp, mean (SD)                                                                       1     119.9 (21.6)     119.9 (21.8)     119.8 (21.3)   0.951
spo2, mean (SD)                                                                      1       97.2 (2.8)       97.1 (2.7)       97.4 (2.8)   0.002
temp_c, mean (SD)                                                                    1       36.2 (5.9)       36.2 (6.0)       36.3 (5.7)   0.447
weight_kg, mean (SD)                                                                 1      88.6 (27.4)      88.9 (27.7)      88.2 (27.0)   0.299
rass, mean (SD)                                                                    445       -2.3 (1.9)       -2.3 (1.8)       -2.3 (1.9)   0.530
midazolam, mean (SD)                                                                 0     36.1 (240.1)     34.6 (261.2)     37.8 (213.4)   0.627
lorazepam, mean (SD)                                                                 0        0.1 (3.7)        0.0 (2.3)        